In [ ]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

In [ ]:

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution,random_solution
from VRP.modify_solution import *
from VRP.verify_solution import valid_path, calculate_path_distance, num_vehicles
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path


import pandas as pd
import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.solver_model import MultiAgentSolverModel
from multi_agent.colaboration_types import ColaborationTypes
from solver_step import SolverStep
from multi_agent.solution_pool import solution_pool_simple
import pandas as pd

In [ ]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()
chosen_route = routes[4]
dists, demand, points = format_input(chosen_route,customer_df,deposit_df)

truck_volume_capacity = 30
truck_weight_capacity = 20000
truck_capacity = (truck_weight_capacity, truck_volume_capacity)

In [ ]:
eval_function = lambda x: 0.3*num_vehicles(x) + calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [ ]:
init_state = random_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),100)

init_step_generator = lambda : SolverStep(random_solution(dists, demand, truck_capacity),eval_function(init_state),100)

# Multi Agents without Q-Learning

In [ ]:
neighbour_function = lambda x: combined_rand_modification(x, len(points), dists)

In [ ]:
solver_step_SA = SA.generic_solver_factory(
    neighbour_function,
    eval_function,
    )

In [ ]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    neighbour_function,
    eval_function,
    taboo_list_size
    )

In [ ]:
step_function_list = [solver_step_SA, solver_step_TS]
agent_labels = ['simulated annealing', 'taboo search']

In [ ]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(
    init_step_generator,
     step_function_list,
     chosen_route,20000,30,
     solution_pool,
     10,
     colaboration_type = ColaborationTypes.FRIENDS,
     agent_labels = agent_labels,
     )

In [ ]:
for _ in range(2):
    sol_model.step()

In [ ]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states.loc[49]['TheGlobalBest'])

In [ ]:
agent_sol_states = sol_model.datacollector.get_agent_vars_dataframe()
print(agent_sol_states)

# Multi Agents with Q-Learning

In [ ]:
from q_learning import *

function_list = [
    lambda x:intra_route_swap(x,len(dists),dists),
    lambda x:inter_route_swap(x,len(dists),dists),
    lambda x:intra_route_shift(x,len(dists),dists),
    lambda x:inter_route_shift(x,len(dists),dists),
    lambda x:two_intra_route_shift(x,len(dists),dists),
    lambda x:two_intra_route_swap(x,len(dists),dists),
    lambda x:remove_smallest_route(x,len(dists),dists),
    lambda x:remove_random_route(x,len(dists),dists),
    lambda x:split_biggest_route(x,len(dists),dists),
    lambda x:split_random_route(x,len(dists),dists)
    ]

q = q_learning_obj(0,init_q_learning_table(len(function_list)),len(function_list), 0.9, 0.1)
neighbour_function = lambda x: q_learning_iteration(x,q,function_list,eval_function)

In [ ]:
solver_step_SA = SA.generic_solver_factory(
    neighbour_function,
    eval_function,
    )

In [ ]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    neighbour_function,
    eval_function,
    taboo_list_size
    )

In [ ]:
step_function_list = [solver_step_SA, solver_step_TS]
agent_labels = ['simulated annealing', 'taboo search']

In [ ]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(
    init_step_generator,
     step_function_list,
     chosen_route,20000,30,
     solution_pool,
     10,
     colaboration_type = ColaborationTypes.FRIENDS,
     QLearn_q = q,
     QLearn_neighbor_function_list = function_list,
     QLearn_eval_function = eval_function,
     agent_labels = agent_labels
     )

In [ ]:
for _ in range(2):
    sol_model.step()